In [14]:
import redis
import json

In [15]:
dataset_path = "../data/datasets/careful"
properties_path = "../data/outputs/careful.json"
metric = "pitch_histogram"

In [ ]:
# FT.CREATE idx:table ON JSON PREFIX 1 cmp: SCHEMA $.sim AS sim NUMERIC $.row_file AS row_file TEXT $.col_file AS col_file TEXT $.metric AS metric TEXT

In [16]:
r = redis.Redis(host="localhost", port=6379, db=0)
properties = {}
with open(properties_path, "r") as f:
    properties = json.load(f)

names = list(properties.keys())
names.sort()
len(names)

3868

In [21]:
print(f"{names[0]} {names[-1]} {metric}")
# FT.CREATE idx:files ON JSON PREFIX 1 doc: SCHEMA $.row_file AS row_file TEXT $.col_file AS col_file TEXT $.metric AS metric TEXT
r.json().get(f"{names[0]}:{names[1]}:{metric}")

20231220-80-01_0000-0008.mid 20240227-76-05_0944-0952.mid pitch_histogram


{'sim': 1.0000000000000002,
 'mutations': {'shift': 0, 'trans': 0},
 'row_file': '20231220-80-01_0000-0008.mid',
 'col_file': '20231220-80-01_0008-0016.mid',
 'metric': 'pitch_histogram'}

In [19]:
r.json().get(f"file:{names[0]}")

{'pitch_histogram_best': '20240227-76-05_0944-0952.mid'}

In [5]:
def scan_keys(r, pattern):
    cursor = 0
    keys = []
    while True:
        cursor, new_keys = r.scan(cursor, match=pattern)
        keys.extend(new_keys)
        if cursor == 0:
            break
    return keys


# Pattern to match
pattern = "20231220-80-01_0000-0008.mid:*:pitch_histogram"

# Get all keys matching the pattern
# matching_keys = scan_keys(r, pattern)
# print(f"Keys matching pattern '{pattern}': {matching_keys}")

In [6]:
def process_json_keys(redis_conn):
    cursor = "0"
    while cursor != 0:
        cursor, keys = redis_conn.scan(cursor=cursor, count=1000)
        for key in keys:
            key_type = redis_conn.execute_command("TYPE", key)
            if key_type == b"ReJSON-RL":
                value = redis_conn.json().get(key)

                if value:
                    row_file, col_file, metric = str(key).split(":")

                    value["row_file"] = row_file[2:]
                    value["col_file"] = col_file
                    value["metric"] = metric

                    # print(f"Key: {key}, Data: {value}")
                    r.json().set(key, "$", value)

            else:
                # Ignore non-JSON objects
                continue
        print(f"finished section {cursor}")


# Call the function
process_json_keys(r)
print("DONE")

finished section 16130048
finished section 11866112
finished section 16461824
finished section 11831296
finished section 436224
finished section 366592
finished section 4175872
finished section 8274944
finished section 6906880
finished section 54272
finished section 10417152
finished section 11521024
finished section 12196864
finished section 80896
finished section 10289664
finished section 15147520
finished section 4624896
finished section 8237568
finished section 15747584
finished section 2517504
finished section 481792
finished section 1014272
finished section 9225728
finished section 5891584
finished section 1037824
finished section 6032896
finished section 1715712
finished section 7413248
finished section 13975040
finished section 9470208
finished section 2269440
finished section 14061824
finished section 14397696
finished section 15354112
finished section 485632
finished section 7395584
finished section 456960
finished section 6538496
finished section 13624576
finished section 74

KeyboardInterrupt: 

In [6]:
query = 'FT.SEARCH idx:sim "20231220-80-01_0000-0008.mid:*:pitch_histogram" SORTBY sim DESC LIMIT 0 10'
result = r.execute_command(query)

# Extract the key with the largest 'sim' value
if len(result) > 1:
    key_with_max_sim = result[1]
    print(f"Key with the largest 'sim' value: {key_with_max_sim}")
else:
    print("No matching keys found.")

ResponseError: Syntax error at offset 29 near mid